In [2]:
# !git clone https://github.com/SZTankWang/ml-weibo-project-.git
%pip install paddlepaddle
%pip install jieba --upgrade
# %pip install jieba
# -tiny==1.6.1

Note: you may need to restart the kernel to use updated packages.
  Using cached jieba-0.42.1.tar.gz (19.2 MB)
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314482 sha256=f21629f89713d278298896f8392ec3666652e8368585720a9a47edfe74e78c70
  Stored in directory: c:\users\mybook\appdata\local\pip\cache\wheels\ca\38\d8\dfdfe73bec1d12026b30cb7ce8da650f3f0ea2cf155ea018ae
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


In [1]:
import jieba
import paddle
import re
import pandas as pd
import time
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor


import math as m
import random as r
import numpy as np

paddle.enable_static()
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持

Paddle enabled successfully......
2021-05-08 22:22:58,291 - DEBUG - Paddle enabled successfully......


In [2]:
df = pd.read_excel("data/raw_data.xlsx",skiprows=1,usecols=["微博正文","点赞数","转发数","评论数"])
df.head()

E:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
E:\Anaconda\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
E:\Anaconda\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,微博正文,点赞数,转发数,评论数
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [3]:
df = df.rename(columns = {'微博正文':'text', '点赞数':'like', '转发数':'comment','评论数':'forward'}, inplace=False)
df.head()

E:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,like,comment,forward
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [4]:
f = open("data/tfidf.txt", 'r',encoding="utf8")
allsentences = f.readlines()

for i in range(len(allsentences)):
  allsentences[i] = allsentences[i].strip('\n')
y = df[['like','comment','forward']]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(allsentences, y, train_size=.7, test_size=.3, random_state=0, shuffle=True, stratify=None)
f.close()

E:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

 
#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer(max_features=100000)
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfTransformer()
#将文本转为词频矩阵并计算tf-idf
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(X_train))
#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
# X_train_weight = np.asarray(tf_idf)
# print(X_train_weight)
# X_train_weight = tf_idf.toarray().astype(float)
 
print(type(tf_idf))    


# X_test_weight = tf_idf.toarray().astype(float)  # 测试集TF-IDF权重矩阵
# X_test_weight.shape 
#print('输出x_train文本向量：')
#print(X_train_weight)
#print('输出x_test文本向量：')
#print(X_test_weight)

print(type(y_train))
# model = LinearRegression().fit(X_train_weight, y_train)
# y_train_pred = model.predict(X_train_weight)
# y_test_pred = model.predict(X_test_weight)

# train_error = sklearn.metrics.mean_squared_error(y_train, y_train_pred)
# test_error = sklearn.metrics.mean_squared_error(y_test, y_test_pred)


# use SGD partial fit
model = MultiOutputRegressor(SGDRegressor(loss='squared_loss'))

count = 0
start = 0
end = 0
while count < (len(X_train)//10000):
    start = count * 10000 
    end = min((count+1) * 10000,len(X_train)-1)
    print( '{} s iteration, start at {}, end at {}\n '.format(count,start,end))
    count += 1
    
    model.partial_fit(tf_idf[start:end].toarray(),y_train[start:end])
    

#对测试集进行tf-idf权重计算
# tf_idf = tf_idf_transformer.transform(vectorizer.transform(X_test))

E:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>
0 s iteration, start at 0, end at 10000
 
1 s iteration, start at 10000, end at 20000
 
2 s iteration, start at 20000, end at 30000
 
3 s iteration, start at 30000, end at 40000
 
4 s iteration, start at 40000, end at 50000
 
5 s iteration, start at 50000, end at 60000
 
6 s iteration, start at 60000, end at 70000
 


In [47]:
y_train_pred = model.predict(tf_idf[:1000].toarray())
train_error = sklearn.metrics.mean_squared_error(y_train[:1000], y_train_pred[:1000])
print(train_error)

E:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1327101982.564471


In [40]:
test_text = "7日 四川 雅安 达瓦更扎 景区 发布 一则 寻人启事 寻找 两位 五一 假期  景区 捡拾 垃圾 游客 景区 表示 两名 游客 送上 川西 最美 雪山 游客 VIP 待遇 本人 户口本 所有 家人 终身 免费 游玩 达瓦更扎 景区 线索 请 联系 0835-5901900"

E:\Anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
test = tf_idf_transformer.transform(vectorizer.transform([test_text])).toarray()
model.predict(test)


array([[14049.4830145 , 12853.92637801,  1497.14544825]])

In [37]:
print(y_test.iloc[0])
print(X_test[0])


like       253
comment    573
forward    672
Name: 54824, dtype: object
各国 治黄 美国 华达州 部分 地区 外 性交易 属于 违法行为 英国 涉及 妓院 活动 认为 犯罪 可能 面临 长达 7年 监禁 日本 扫黄 重点 欺骗 迫使 卖淫 经营 色情业 行为 提供 资金 看看 国家 管理 情色业 httptcn8Fj7yNq 原图
